# Array

In [1]:
import copy
import json
import sys

from konfoo import *

## YAML Support

In [2]:
import oyaml as yaml

In [3]:
def to_yaml(container, *attributes, **options):
    flow_style = options.pop('flow_style', False)
    return yaml.safe_dump(container.view_fields(*attributes, **options), 
                          stream=sys.stdout, 
                          default_flow_style=flow_style)

## Create an Array

In [4]:
array = Array(template=Byte, capacity=4)
array = Array(Byte, 4)

In [5]:
array.to_list()

[('Array[0]', '0x0'),
 ('Array[1]', '0x0'),
 ('Array[2]', '0x0'),
 ('Array[3]', '0x0')]

In [6]:
array.to_csv()

[{'id': 'Array[0]', 'value': '0x0'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[2]', 'value': '0x0'},
 {'id': 'Array[3]', 'value': '0x0'}]

In [7]:
array.to_json()

'["0x0", "0x0", "0x0", "0x0"]'

In [8]:
to_yaml(array, flow_style=False)

- '0x0'
- '0x0'
- '0x0'
- '0x0'


## Create an Array of Pointers

Define a concrete `data object pointer` class and use the **class** as the `array element` *template*.

In [9]:
class BytePointer(Pointer):
    def __init__(self):
        super().__init__(Byte())

In [10]:
array = Array(BytePointer, 2)

In [11]:
array.to_list(nested=True)

[('Array[0]', '0x0'),
 ('Array[0].data', '0x0'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x0')]

In [12]:
array.to_csv(nested=True)

[{'id': 'Array[0]', 'value': '0x0'},
 {'id': 'Array[0].data', 'value': '0x0'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[1].data', 'value': '0x0'}]

In [13]:
array.to_json(nested=True)

'[{"value": "0x0", "data": "0x0"}, {"value": "0x0", "data": "0x0"}]'

In [14]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x0'
  data: '0x0'
- value: '0x0'
  data: '0x0'


In [15]:
array[0].value = 1

In [16]:
array[0].data.value = 2

In [17]:
array.to_list(nested=True)

[('Array[0]', '0x1'),
 ('Array[0].data', '0x2'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x0')]

In [18]:
array.to_csv(nested=True)

[{'id': 'Array[0]', 'value': '0x1'},
 {'id': 'Array[0].data', 'value': '0x2'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[1].data', 'value': '0x0'}]

In [19]:
array.to_json(nested=True)

'[{"value": "0x1", "data": "0x2"}, {"value": "0x0", "data": "0x0"}]'

In [20]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x1'
  data: '0x2'
- value: '0x0'
  data: '0x0'


## Create an Array with nested Members

### The Wrong Way: Created with an Instance

In [21]:
array = Array(Pointer(Byte()), 2)

In [22]:
array.to_list(nested=True)

[('Array[0]', '0x0'),
 ('Array[0].data', '0x0'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x0')]

In [23]:
array.to_csv(nested=True)

[{'id': 'Array[0]', 'value': '0x0'},
 {'id': 'Array[0].data', 'value': '0x0'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[1].data', 'value': '0x0'}]

In [24]:
array.to_json(nested=True)

'[{"value": "0x0", "data": "0x0"}, {"value": "0x0", "data": "0x0"}]'

In [25]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x0'
  data: '0x0'
- value: '0x0'
  data: '0x0'


In [26]:
array[0].value = 1

In [27]:
array[0].data.value = 2

In [28]:
array.to_list(nested=True)

[('Array[0]', '0x1'),
 ('Array[0].data', '0x2'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x2')]

In [29]:
array.to_csv(nested=True)

[{'id': 'Array[0]', 'value': '0x1'},
 {'id': 'Array[0].data', 'value': '0x2'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[1].data', 'value': '0x2'}]

In [30]:
array.to_json(nested=True)

'[{"value": "0x1", "data": "0x2"}, {"value": "0x0", "data": "0x2"}]'

In [31]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x1'
  data: '0x2'
- value: '0x0'
  data: '0x2'


### The Right Way: Created with a Factory

Define a `factory` for the `array element` *template* of the `array`.

In [32]:
class FieldPointerFactory:
    """ A factory class to produce a pointer array element to any field. """
    
    def __init__(self, template):
        # Data object: field template (instance or class).
        self.template = template
        
    def _create_data_object(self):
        """ Produces the data object attached to the pointer. """
        if is_field(self.template):
            # Copy data object instance from instance template
            return copy.copy(self.template)
        elif callable(self.template):
            # Create data object instance from class template
            data_object = self.template()
            if is_field(data_object):
                return data_object
            else:
                raise FactoryTypeError(self, self.template, data_object)
        else:
            raise MemberTypeError(self, self.template)
                                       
    def __call__(self):                                                      
        """ Produces the array element. """
        return Pointer(self._create_data_object())

In [33]:
array = Array(FieldPointerFactory(Byte()), 2)

In [34]:
array.to_list(nested=True)

[('Array[0]', '0x0'),
 ('Array[0].data', '0x0'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x0')]

In [35]:
array.to_csv(nested=True)

[{'id': 'Array[0]', 'value': '0x0'},
 {'id': 'Array[0].data', 'value': '0x0'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[1].data', 'value': '0x0'}]

In [36]:
array.to_json(nested=True)

'[{"value": "0x0", "data": "0x0"}, {"value": "0x0", "data": "0x0"}]'

In [37]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x0'
  data: '0x0'
- value: '0x0'
  data: '0x0'


In [38]:
array[0].value = 1

In [39]:
array[0].data.value = 2

In [40]:
array.to_list(nested=True)

[('Array[0]', '0x1'),
 ('Array[0].data', '0x2'),
 ('Array[1]', '0x0'),
 ('Array[1].data', '0x0')]

In [41]:
array.to_json(nested=True)

'[{"value": "0x1", "data": "0x2"}, {"value": "0x0", "data": "0x0"}]'

In [42]:
to_yaml(array, nested=True, flow_style=False)

- value: '0x1'
  data: '0x2'
- value: '0x0'
  data: '0x0'


## Resize an Array

In [43]:
array = Array(Byte())

In [44]:
array.to_list()

[]

In [45]:
array.to_csv()

[]

In [46]:
array.to_json()

'[]'

In [47]:
to_yaml(array, flow_style=False)

[]


In [48]:
array.resize(4)

In [49]:
array.to_list()

[('Array[0]', '0x0'),
 ('Array[1]', '0x0'),
 ('Array[2]', '0x0'),
 ('Array[3]', '0x0')]

In [50]:
array.to_csv()

[{'id': 'Array[0]', 'value': '0x0'},
 {'id': 'Array[1]', 'value': '0x0'},
 {'id': 'Array[2]', 'value': '0x0'},
 {'id': 'Array[3]', 'value': '0x0'}]

In [51]:
array.to_json()

'["0x0", "0x0", "0x0", "0x0"]'

In [52]:
to_yaml(array, flow_style=False)

- '0x0'
- '0x0'
- '0x0'
- '0x0'


## Initialize a Array

In [53]:
array.initialize_fields([1, 2])

In [54]:
array.to_list()

[('Array[0]', '0x1'),
 ('Array[1]', '0x2'),
 ('Array[2]', '0x1'),
 ('Array[3]', '0x2')]

In [55]:
array.to_csv()

[{'id': 'Array[0]', 'value': '0x1'},
 {'id': 'Array[1]', 'value': '0x2'},
 {'id': 'Array[2]', 'value': '0x1'},
 {'id': 'Array[3]', 'value': '0x2'}]

In [56]:
array.to_json()

'["0x1", "0x2", "0x1", "0x2"]'

In [57]:
to_yaml(array, flow_style=False)

- '0x1'
- '0x2'
- '0x1'
- '0x2'


## Display an Array

In [58]:
array

[Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x1'), Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x2'), Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x1'), Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x2')]

## Metadata of an Array

In [59]:
array.describe()

{'class': 'Array',
 'name': 'Array',
 'size': 4,
 'type': 'Array',
 'member': [{'address': 0,
   'alignment': [1, 0],
   'class': 'Byte',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'Array[0]',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': '0x1'},
  {'address': 0,
   'alignment': [1, 0],
   'class': 'Byte',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'Array[1]',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': '0x2'},
  {'address': 0,
   'alignment': [1, 0],
   'class': 'Byte',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'Array[2]',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': '0x1'},
  {'address': 0,
   'alignment': [1, 0],
   'class': 'Byte',
   'index': [0, 0],
   'max': 255,
   'min': 0,
   'name': 'Array[3]',
   'order': 'auto',
   'signed': False,
   'size': 8,
   'type': 'Field',
   'value': '0x2'}]}

In [60]:
json.dump(array.describe(), sys.stdout, indent=2)

{
  "class": "Array",
  "name": "Array",
  "size": 4,
  "type": "Array",
  "member": [
    {
      "address": 0,
      "alignment": [
        1,
        0
      ],
      "class": "Byte",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "Array[0]",
      "order": "auto",
      "signed": false,
      "size": 8,
      "type": "Field",
      "value": "0x1"
    },
    {
      "address": 0,
      "alignment": [
        1,
        0
      ],
      "class": "Byte",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "Array[1]",
      "order": "auto",
      "signed": false,
      "size": 8,
      "type": "Field",
      "value": "0x2"
    },
    {
      "address": 0,
      "alignment": [
        1,
        0
      ],
      "class": "Byte",
      "index": [
        0,
        0
      ],
      "max": 255,
      "min": 0,
      "name": "Array[2]",
      "order": "auto",
      "signed": false,
      "size": 8,
 

In [61]:
d3flare_json(array.describe(), sys.stdout, indent=2)

{
  "class": "Array",
  "name": "Array",
  "children": [
    {
      "class": "Byte",
      "name": "Array[0]",
      "size": 8,
      "value": "0x1"
    },
    {
      "class": "Byte",
      "name": "Array[1]",
      "size": 8,
      "value": "0x2"
    },
    {
      "class": "Byte",
      "name": "Array[2]",
      "size": 8,
      "value": "0x1"
    },
    {
      "class": "Byte",
      "name": "Array[3]",
      "size": 8,
      "value": "0x2"
    }
  ]
}

## Size of an Array

In [62]:
array.container_size()

(4, 0)

In [63]:
num_of_bytes, num_of_remaining_bits = array.container_size()

In [64]:
num_of_bytes

4

In [65]:
num_of_remaining_bits

0

## Indexing

In [66]:
array.to_list('index')

[('Array[0]', Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Array[1]', Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Array[2]', Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Array[3]', Index(byte=0, bit=0, address=0, base_address=0, update=False))]

In [67]:
array.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'))

[{'id': 'Array[0]', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Array[1]', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Array[2]', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Array[3]', 'index': 0, 'offset': 0, 'address': 0}]

In [68]:
array.to_json('index')

'[[0, 0, 0, 0, false], [0, 0, 0, 0, false], [0, 0, 0, 0, false], [0, 0, 0, 0, false]]'

In [69]:
array.index_fields(index=Index())

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [70]:
array.index_fields()

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [71]:
array.to_list('index')

[('Array[0]', Index(byte=0, bit=0, address=0, base_address=0, update=False)),
 ('Array[1]', Index(byte=1, bit=0, address=1, base_address=0, update=False)),
 ('Array[2]', Index(byte=2, bit=0, address=2, base_address=0, update=False)),
 ('Array[3]', Index(byte=3, bit=0, address=3, base_address=0, update=False))]

In [72]:
array.to_csv('index.byte', 'index.bit', 'index.address', fieldnames=('id', 'index', 'offset', 'address'))

[{'id': 'Array[0]', 'index': 0, 'offset': 0, 'address': 0},
 {'id': 'Array[1]', 'index': 1, 'offset': 0, 'address': 1},
 {'id': 'Array[2]', 'index': 2, 'offset': 0, 'address': 2},
 {'id': 'Array[3]', 'index': 3, 'offset': 0, 'address': 3}]

In [73]:
array.to_json('index')

'[[0, 0, 0, 0, false], [1, 0, 1, 0, false], [2, 0, 2, 0, false], [3, 0, 3, 0, false]]'

## De-Serializing

In [74]:
array.deserialize(bytes.fromhex('01020304f00f00'))

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [75]:
array.to_list()

[('Array[0]', '0x1'),
 ('Array[1]', '0x2'),
 ('Array[2]', '0x3'),
 ('Array[3]', '0x4')]

In [76]:
array.to_csv()

[{'id': 'Array[0]', 'value': '0x1'},
 {'id': 'Array[1]', 'value': '0x2'},
 {'id': 'Array[2]', 'value': '0x3'},
 {'id': 'Array[3]', 'value': '0x4'}]

In [77]:
array.to_json()

'["0x1", "0x2", "0x3", "0x4"]'

In [78]:
to_yaml(array, flow_style=False)

- '0x1'
- '0x2'
- '0x3'
- '0x4'


## Serializing

In [79]:
bytestream = bytearray()
bytestream.hex()

''

In [80]:
array.serialize(bytestream)

Index(byte=4, bit=0, address=4, base_address=0, update=False)

In [81]:
bytes(array).hex()

'01020304'

In [82]:
bytestream.hex()

'01020304'

## Array Members

### Number of Members

In [83]:
len(array)

4

### Access a Member

In [84]:
array[0]

Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x1')

### Access the Attributes of a Member Field

In [85]:
array[0].name

'Byte'

In [86]:
array[0].value

'0x1'

In [87]:
array[0].bit_size

8

In [88]:
array[0].alignment

Alignment(byte_size=1, bit_offset=0)

In [89]:
array[0].alignment.byte_size

1

In [90]:
array[0].alignment.bit_offset

0

In [91]:
array[0].byte_order

Byteorder.auto = 'auto'

In [92]:
array[0].byte_order.value

'auto'

In [93]:
array[0].index

Index(byte=0, bit=0, address=0, base_address=0, update=False)

In [94]:
array[0].index.byte

0

In [95]:
array[0].index.bit

0

In [96]:
array[0].index.address

0

In [97]:
array[0].index.base_address

0

In [98]:
array[0].index.update

False

### List the Members

In [99]:
[member.item_type for member in array]

[ItemClass.Byte = 42,
 ItemClass.Byte = 42,
 ItemClass.Byte = 42,
 ItemClass.Byte = 42]

## Array Fields

### View Field Attributes

In [100]:
array.view_fields()

['0x1', '0x2', '0x3', '0x4']

In [101]:
array.view_fields('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order.name')

[{'name': 'Byte',
  'bit_size': 8,
  'value': '0x1',
  'index': Index(byte=0, bit=0, address=0, base_address=0, update=False),
  'alignment': Alignment(byte_size=1, bit_offset=0),
  'byte_order.name': 'auto'},
 {'name': 'Byte',
  'bit_size': 8,
  'value': '0x2',
  'index': Index(byte=1, bit=0, address=1, base_address=0, update=False),
  'alignment': Alignment(byte_size=1, bit_offset=0),
  'byte_order.name': 'auto'},
 {'name': 'Byte',
  'bit_size': 8,
  'value': '0x3',
  'index': Index(byte=2, bit=0, address=2, base_address=0, update=False),
  'alignment': Alignment(byte_size=1, bit_offset=0),
  'byte_order.name': 'auto'},
 {'name': 'Byte',
  'bit_size': 8,
  'value': '0x4',
  'index': Index(byte=3, bit=0, address=3, base_address=0, update=False),
  'alignment': Alignment(byte_size=1, bit_offset=0),
  'byte_order.name': 'auto'}]

### View as a JSON string

In [102]:
array.to_json()

'["0x1", "0x2", "0x3", "0x4"]'

In [103]:
print(array.to_json(indent=2))

[
  "0x1",
  "0x2",
  "0x3",
  "0x4"
]


In [104]:
array.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

'[{"name": "Byte", "bit_size": 8, "value": "0x1", "index": [0, 0, 0, 0, false], "alignment": [1, 0], "byte_order": "auto"}, {"name": "Byte", "bit_size": 8, "value": "0x2", "index": [1, 0, 1, 0, false], "alignment": [1, 0], "byte_order": "auto"}, {"name": "Byte", "bit_size": 8, "value": "0x3", "index": [2, 0, 2, 0, false], "alignment": [1, 0], "byte_order": "auto"}, {"name": "Byte", "bit_size": 8, "value": "0x4", "index": [3, 0, 3, 0, false], "alignment": [1, 0], "byte_order": "auto"}]'

In [105]:
print(array.to_json('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order', indent=2))

[
  {
    "name": "Byte",
    "bit_size": 8,
    "value": "0x1",
    "index": [
      0,
      0,
      0,
      0,
      false
    ],
    "alignment": [
      1,
      0
    ],
    "byte_order": "auto"
  },
  {
    "name": "Byte",
    "bit_size": 8,
    "value": "0x2",
    "index": [
      1,
      0,
      1,
      0,
      false
    ],
    "alignment": [
      1,
      0
    ],
    "byte_order": "auto"
  },
  {
    "name": "Byte",
    "bit_size": 8,
    "value": "0x3",
    "index": [
      2,
      0,
      2,
      0,
      false
    ],
    "alignment": [
      1,
      0
    ],
    "byte_order": "auto"
  },
  {
    "name": "Byte",
    "bit_size": 8,
    "value": "0x4",
    "index": [
      3,
      0,
      3,
      0,
      false
    ],
    "alignment": [
      1,
      0
    ],
    "byte_order": "auto"
  }
]


### Write to a JSON file

In [106]:
array.write_json('Array.json')

### List the Field Items

In [107]:
array.field_items()

[('[0]',
  Byte(index=Index(byte=0, bit=0, address=0, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x1')),
 ('[1]',
  Byte(index=Index(byte=1, bit=0, address=1, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x2')),
 ('[2]',
  Byte(index=Index(byte=2, bit=0, address=2, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x3')),
 ('[3]',
  Byte(index=Index(byte=3, bit=0, address=3, base_address=0, update=False), alignment=Alignment(byte_size=1, bit_offset=0), bit_size=8, value='0x4'))]

### List the Field Attributes

In [108]:
array.to_list()

[('Array[0]', '0x1'),
 ('Array[1]', '0x2'),
 ('Array[2]', '0x3'),
 ('Array[3]', '0x4')]

In [109]:
array.to_list('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

[('Array[0]',
  ('Byte',
   8,
   '0x1',
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Array[1]',
  ('Byte',
   8,
   '0x2',
   Index(byte=1, bit=0, address=1, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Array[2]',
  ('Byte',
   8,
   '0x3',
   Index(byte=2, bit=0, address=2, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto')),
 ('Array[3]',
  ('Byte',
   8,
   '0x4',
   Index(byte=3, bit=0, address=3, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto'))]

In [110]:
array.to_dict()

{'Array': {'[0]': '0x1', '[1]': '0x2', '[2]': '0x3', '[3]': '0x4'}}

In [111]:
array.to_dict('name', 'bit_size', 'value', 'index', 'alignment', 'byte_order')

{'Array': {'[0]': ('Byte',
   8,
   '0x1',
   Index(byte=0, bit=0, address=0, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto'),
  '[1]': ('Byte',
   8,
   '0x2',
   Index(byte=1, bit=0, address=1, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto'),
  '[2]': ('Byte',
   8,
   '0x3',
   Index(byte=2, bit=0, address=2, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto'),
  '[3]': ('Byte',
   8,
   '0x4',
   Index(byte=3, bit=0, address=3, base_address=0, update=False),
   Alignment(byte_size=1, bit_offset=0),
   Byteorder.auto = 'auto')}}

In [112]:
array.to_csv()

[{'id': 'Array[0]', 'value': '0x1'},
 {'id': 'Array[1]', 'value': '0x2'},
 {'id': 'Array[2]', 'value': '0x3'},
 {'id': 'Array[3]', 'value': '0x4'}]

In [113]:
array.to_csv('name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'))

[{'id': 'Array[0]', 'type': 'Byte', 'size': 8, 'value': '0x1'},
 {'id': 'Array[1]', 'type': 'Byte', 'size': 8, 'value': '0x2'},
 {'id': 'Array[2]', 'type': 'Byte', 'size': 8, 'value': '0x3'},
 {'id': 'Array[3]', 'type': 'Byte', 'size': 8, 'value': '0x4'}]

### Write the Field Attributes to a `.csv` File

In [114]:
array.write_csv('Array.csv', 'name', 'bit_size', 'value', fieldnames=('id', 'type', 'size', 'value'))

### Save the Field Attributes to an `.ini` File

In [115]:
array.save('Array.ini')

### Load the Field Value from an `.ini` File

In [116]:
array.load('Array.ini')

[Array]
Array[0] = 0x1
Array[1] = 0x2
Array[2] = 0x3
Array[3] = 0x4
